# GOES EUV Comparisons

Compare overlapping EUV observations for GOES 8 through 18.
- Determin how well the fluxes match by fitting a line to pairs
    - Nominal Slope: 1
    - Nominal Intercept: 0

Identify which pairs most closely match to GOES R-Series (16,17,18). These satellites can be used to train with real-time data for one an operational models. 

In [2]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
import numpy.ma as ma

from netCDF4 import Dataset



In [3]:
fn = "D:\data\GOES\EUVS\sci_geuv-l2-avg1m_g15_s20100325_e20200304_v5-0-0.nc"

f = Dataset(fn)
print(f)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: ACDD-1.3, Spase v2.2.6
    title: GOES 13-15 L2 EUVS 1-min-averaged irradiances and flags.
    summary: The GOES 13-15 Extreme Ultraviolet Sensor (EUVS) measured EUV irradiances in 5 broadband channels. For GOES 13 and 15, these were channels A (5-15 nm), B (25-34 nm), C(two bands in 17-67 nm), D(three bands in 17-84 nm) and E(118-127 nm). GOES-14 measured the A and B channels twice as well as the E channel. This product provides 1-minute average irradiance values and flags for Channels B and E and the relevant 1-nm bandpasses for the 30.4 and 121-nm spectral lines (He II and Lyman alpha).
    keywords: NumericalData.MeasurementType.Irradiance
    keywords_vocabulary: SPASE: Space Physics Archive Search and Extract Data Model version 2.2.6, GCMD: NASA Global Change Master Directory (GCMD) Earth Science Keywords version 8.5
    naming_authority: gov.nesdis.noaa
    history: See algorit

In [9]:
irr = ma.getdata(f.variables['irr_1216_1nm'][:])
dt = ma.getdata(f.variables['time'][:])

dt_shift = pd.to_datetime('2000-01-01 12:00:00').value*1E-9    

In [10]:
g_df = pd.DataFrame()
g_df['irr_1216'] = irr
g_df['DateTime'] = pd.to_datetime(dt + dt_shift, unit='s') 

In [22]:
bd = g_df['irr_1216'] < 0
g_df.loc[bd,'irr_1216'] = np.NaN

In [11]:
f.close()

In [30]:
fn = 'D:\\data\\SatDensities\\satdrag_database_grace_B_v3.hdf5'
database = pd.read_hdf(fn)

In [31]:
g_tol = pd.Timedelta('31 seconds')
database = pd.merge_asof(left=database,right=g_df,left_on='DateTime',right_on='DateTime',
                             direction='nearest',tolerance=g_tol)

In [32]:
database.head()

,DateTime,DateTime_omni,Year,DOY,Hour,Minute,IMF_id,SW_id,IMF_pt,SW_pt,...,storm,storm phase,mlat,mlon,mlt,xrsa_flux_g11,xrsb_flux_g11,xrsa_flux_g15,xrsb_flux_g15,irr_1216
0,2002-04-04 00:50:00,2002-04-04 00:50:00,2002,94,0,50,71.0,71.0,5.0,5.0,...,1,2,-71.403237,-126.574355,11.594145,NaN,NaN,NaN,NaN,NaN
1,2002-04-04 00:55:00,2002-04-04 00:55:00,2002,94,0,55,71.0,71.0,5.0,4.0,...,1,2,-52.080051,-136.756996,10.997453,NaN,NaN,NaN,NaN,NaN
2,2002-04-04 01:00:00,2002-04-04 01:00:00,2002,94,1,0,71.0,71.0,5.0,5.0,...,1,2,-33.379533,-141.350071,10.773452,NaN,NaN,NaN,NaN,NaN
3,2002-04-04 01:05:00,2002-04-04 01:05:00,2002,94,1,5,71.0,71.0,5.0,5.0,...,1,2,-18.508417,-144.383796,10.653469,NaN,NaN,NaN,NaN,NaN
4,2002-04-04 01:10:00,2002-04-04 01:10:00,2002,94,1,10,71.0,71.0,5.0,5.0,...,1,2,18.645368,-146.158078,10.617515,NaN,NaN,NaN,NaN,NaN


In [33]:
database.to_hdf(fn,key='database', mode='w', format='table', complevel=7)

In [28]:
dens_col = ['dens_x']
columns = ['irr_1216_y','dens_x']

cor_plot = pd.DataFrame()
cor_plot['All'] = database[columns].dropna().corr()[dens_col]

In [29]:
cor_plot

,All
irr_1216_y,0.621531
dens_x,1.000000


In [17]:
g_df

,irr_1216,DateTime
0,-9999.000000,2010-03-25 00:00:00
1,-9999.000000,2010-03-25 00:01:00
2,-9999.000000,2010-03-25 00:02:00
3,-9999.000000,2010-03-25 00:03:00
4,-9999.000000,2010-03-25 00:04:00
...,...,...
5231515,0.005846,2020-03-04 23:55:00
5231516,0.005833,2020-03-04 23:56:00
5231517,0.005836,2020-03-04 23:57:00
5231518,0.005849,2020-03-04 23:58:00


In [20]:
pd.to_datetime(f.variables['time'].units)

DateParseError: Unknown datetime string format, unable to parse: seconds since 2000-01-01 12:00:00 UTC, at position 0

In [3]:
a = pd.to_datetime('2000-01-01 12:00:00')

In [4]:
a

Timestamp('2000-01-01 12:00:00')

In [5]:
a.value

946728000000000000

In [15]:
pd.to_datetime(a.value*1E-9, unit='s')

Timestamp('2000-01-01 12:00:00')

In [6]:
pd.to_datetime(f.variables['time'][-1]+a.value)

Timestamp('2000-01-01 12:00:00.636638336')

In [18]:
pd.to_datetime(ma.getdata(f.variables['time'][0])+(a.value*1E-9),unit='s')

Timestamp('2010-03-25 00:00:00')

In [10]:
g_df

,irr_1216,DateTime
0,-9999.000000,1970-01-01 00:00:00.322747200
1,-9999.000000,1970-01-01 00:00:00.322747260
2,-9999.000000,1970-01-01 00:00:00.322747320
3,-9999.000000,1970-01-01 00:00:00.322747380
4,-9999.000000,1970-01-01 00:00:00.322747440
...,...,...
5231515,0.005846,1970-01-01 00:00:00.636638100
5231516,0.005833,1970-01-01 00:00:00.636638160
5231517,0.005836,1970-01-01 00:00:00.636638220
5231518,0.005849,1970-01-01 00:00:00.636638280
